# 变形 笔记

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/table.csv')
df.head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


## 一、透视表

### 1.pivot
#### 一般状态下，数据在DataFrame会以压缩（stacked）状态存放，例如上面的Gender，两个类别被叠在一列中，pivot函数可将某一列作为新的cols：

In [2]:
df.pivot(index='ID',columns='Gender',values='Height').head()

Gender,F,M
ID,,
1101,NaN,173.0
1102,192.0,NaN
1103,NaN,186.0
1104,167.0,NaN
1105,159.0,NaN


#### 然而pivot函数具有很强的局限性，除了功能上较少之外，还不允许values中出现重复的行列索引对（pair），例如下面的语句就会报错：

In [4]:
# df.pivot(index='School',columns='Gender',values='Height').head()
# Index contains duplicate entries, cannot reshape


#### 因此，大多数情况会使用pivot_table

### 2. pivot_table

In [6]:
pd.pivot_table(df,index='ID',columns='Gender',values='Height').head()

Gender,F,M
ID,,
1101,NaN,173.0
1102,192.0,NaN
1103,NaN,186.0
1104,167.0,NaN
1105,159.0,NaN


In [7]:
# 速度比不上pivot，由于功能更多
%timeit df.pivot(index='ID',columns='Gender',values='Height')
%timeit pd.pivot_table(df,index='ID',columns='Gender',values='Height')

1.55 ms ± 48.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
6.9 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### 常用参数

#### 1) aggfunc: 对组内进行聚合统计，可传入各类函数，默认为‘mean’

In [11]:
pd.pivot_table(df,index='School',columns='Gender',values='Height',
               aggfunc=['mean','sum']).head()

mean               sum      
Gender           F           M     F     M
School                                    
S_1     173.125000  178.714286  1385  1251
S_2     173.727273  172.000000  1911  1548

In [13]:
df.pivot_table(index='School',columns='Gender',values='Height')
# 不做聚合统计，默认为均值

Gender,F,M
School,,
S_1,173.125000,178.714286
S_2,173.727273,172.000000


#### 2) margins：汇总边际状态

In [15]:
pd.pivot_table(df,index='School',columns='Gender',
               values='Height',aggfunc=['mean','sum'],
               margins=True).head()
#margins_name可以设置名字，默认为'All'
#汇总边际状态也就是将columns下的所有情况做一个聚合

mean                           sum            
Gender           F           M         All     F     M   All
School                                                      
S_1     173.125000  178.714286  175.733333  1385  1251  2636
S_2     173.727273  172.000000  172.950000  1911  1548  3459
All     173.473684  174.937500  174.142857  3296  2799  6095

In [17]:
pd.pivot_table(df,index=['School','Class'],
               columns=['Gender','Address'],
               values=['Height','Weight'])
# 哇哦。value也可以多级嘛

Height                                                        \
Gender              F                                                     M   
Address      street_1 street_2 street_4 street_5 street_6 street_7 street_1   
School Class                                                                  
S_1    C_1        NaN    179.5    159.0      NaN      NaN      NaN    173.0   
       C_2        NaN      NaN    176.0    162.0    167.0      NaN      NaN   
       C_3      175.0      NaN      NaN    187.0      NaN      NaN      NaN   
S_2    C_1        NaN      NaN      NaN    159.0    161.0      NaN      NaN   
       C_2        NaN      NaN      NaN      NaN      NaN    188.5    175.0   
       C_3        NaN      NaN    157.0      NaN    164.0    190.0      NaN   
       C_4        NaN    176.0      NaN      NaN    175.5      NaN      NaN   

                                         ...   Weight                    \
Gender                                   ...        F                     
Address      street_2 street_4 street_5  ... street_4 street_5 street_6   
School Class                             ...                              
S_1    C_1      186.0      NaN      NaN  ...     64.0      NaN      NaN   
       C_2        NaN      NaN    188.0  ...     94.0     63.0     63.0   
       C_3      195.0    161.0      NaN  ...      NaN     69.0      NaN   
S_2    C_1        NaN    163.5      NaN  ...      NaN     97.0     61.0   
       C_2        NaN    155.0    193.0  ...      NaN      NaN      NaN   
       C_3        NaN    187.0    171.0  ...     78.0      NaN     81.0   
       C_4        NaN      NaN      NaN  ...      NaN      NaN     57.0   

                                                                             
Gender                       M                                               
Address      street_7 street_1 street_2 street_4 street_5 street_6 street_7  
School Class                                                                 
S_1    C_1        NaN     63.0     82.0      NaN      NaN      NaN      NaN  
       C_2        NaN      NaN      NaN      NaN     68.0     53.0      NaN  
       C_3        NaN      NaN     70.0     68.0      NaN      NaN     82.0  
S_2    C_1        NaN      NaN      NaN     71.0      NaN      NaN     84.0  
       C_2       76.5     74.0      NaN     91.0    100.0      NaN      NaN  
       C_3       99.0      NaN      NaN     73.0     88.0      NaN      NaN  
       C_4        NaN      NaN      NaN      NaN      NaN      NaN     82.0  

[7 rows x 24 columns]

### 3. crosstab（交叉表）

交叉表是一种特殊的透视表，典型的用途如分组统计，如现在想要统计关于街道和性别分组的频数：

In [19]:
pd.crosstab(index=df['Address'],columns=df['Gender'])

Gender,F,M
Address,,
street_1,1,2
street_2,4,2
street_4,3,5
street_5,3,3
street_6,5,1
street_7,3,3


交叉表的功能也很强大（但目前还不支持多级分组）

以下是一些重要参数

1）values和aggfunc：分组对某些数据进行聚合操作，这两个参数必须成对出现

In [22]:
pd.crosstab(index=df['Address'],columns=df['Gender'],
            values=np.random.randint(1,20,df.shape[0]),aggfunc='min')

Gender,F,M
Address,,
street_1,10,7
street_2,4,10
street_4,11,2
street_5,1,15
street_6,1,2
street_7,5,4


In [21]:
#默认参数等于如下方法：
pd.crosstab(index=df['Address'],columns=df['Gender'],values=1,aggfunc='count')

Gender,F,M
Address,,
street_1,1,2
street_2,4,2
street_4,3,5
street_5,3,3
street_6,5,1
street_7,3,3


2）除了边际参数margins外，还引入了normalize参数，可选'all','index','columns'参数值

In [23]:
# 边际参数
pd.crosstab(index=df['Address'],columns=df['Gender'],
            margins=True)

Gender,F,M,All
Address,,,
street_1,1,2,3
street_2,4,2,6
street_4,3,5,8
street_5,3,3,6
street_6,5,1,6
street_7,3,3,6
All,19,16,35


In [25]:
# normalize
pd.crosstab(index=df['Address'],columns=df['Gender'],
            normalize='index',margins=True) #行总和为1

Gender,F,M
Address,,
street_1,0.333333,0.666667
street_2,0.666667,0.333333
street_4,0.375000,0.625000
street_5,0.500000,0.500000
street_6,0.833333,0.166667
street_7,0.500000,0.500000
All,0.542857,0.457143


In [27]:
pd.crosstab(index=df['Address'],columns=df['Gender'],
            normalize='columns',margins=True) #列总和为1

Gender,F,M,All
Address,,,
street_1,0.052632,0.1250,0.085714
street_2,0.210526,0.1250,0.171429
street_4,0.157895,0.3125,0.228571
street_5,0.157895,0.1875,0.171429
street_6,0.263158,0.0625,0.171429
street_7,0.157895,0.1875,0.171429


In [29]:
pd.crosstab(index=df['Address'],columns=df['Gender'],
            normalize='all',margins=True) #总和为1

Gender,F,M,All
Address,,,
street_1,0.028571,0.057143,0.085714
street_2,0.114286,0.057143,0.171429
street_4,0.085714,0.142857,0.228571
street_5,0.085714,0.085714,0.171429
street_6,0.142857,0.028571,0.171429
street_7,0.085714,0.085714,0.171429
All,0.542857,0.457143,1.000000


## 二、其他变形方法

### 1.melt

#### melt函数可以认为是pivot函数的逆操作，将unstacked状态的数据，压缩成stacked，使“宽”的DataFrame变“窄”

In [31]:
df_m = df[['ID','Gender','Math']]
df_m.head()

,ID,Gender,Math
0,1101,M,34.0
1,1102,F,32.5
2,1103,M,87.2
3,1104,F,80.4
4,1105,F,84.8


In [34]:
pivoted = df.pivot(index='ID',columns='Gender',values='Math')
pivoted.head()

Gender,F,M
ID,,
1101,NaN,34.0
1102,32.5,NaN
1103,NaN,87.2
1104,80.4,NaN
1105,84.8,NaN


In [36]:
result = pivoted.reset_index().melt(id_vars=['ID'],
                                    value_vars=['F','M'],value_name='Math')\
                     .dropna().set_index('ID').sort_index()
result.head()

,Gender,Math
ID,,
1101,M,34.0
1102,F,32.5
1103,M,87.2
1104,F,80.4
1105,F,84.8


In [38]:
#检验是否与展开前的df相同
result.equals(df_m.set_index('ID'))

True

### 2.压缩与展开

#### 1）stack：这是最基础的变形函数，总共只有两个参数：level和dropna
#### 宽变长

In [40]:
df_s = pd.pivot_table(df,index=['Class','ID'],columns='Gender',values=['Height','Weight'])
df_s.groupby('Class').mean()

Height        Weight      
Gender       F      M      F     M
Class                             
C_1     167.60  172.0   75.2  74.2
C_2     176.40  174.2   74.6  77.2
C_3     174.60  180.4   76.8  76.2
C_4     175.75  166.0   65.0  82.0

In [44]:
df_s.groupby('Class').head(2)

Height        Weight      
Gender          F      M      F     M
Class ID                             
C_1   1101    NaN  173.0    NaN  63.0
      1102  192.0    NaN   73.0   NaN
C_2   1201    NaN  188.0    NaN  68.0
      1202  176.0    NaN   94.0   NaN
C_3   1301    NaN  161.0    NaN  68.0
      1302  175.0    NaN   57.0   NaN
C_4   2401  192.0    NaN   62.0   NaN
      2402    NaN  166.0    NaN  82.0

In [46]:
df_stacked = df_s.stack()
df_stacked.groupby('Class').head(2)

Height  Weight
Class ID   Gender                
C_1   1101 M        173.0    63.0
      1102 F        192.0    73.0
C_2   1201 M        188.0    68.0
      1202 F        176.0    94.0
C_3   1301 M        161.0    68.0
      1302 F        175.0    57.0
C_4   2401 F        192.0    62.0
      2402 M        166.0    82.0

stack函数可以看做将横向的索引放到纵向，因此功能类似与melt，参数level可指定变化的列索引是哪一层（或哪几层，需要列表）

In [50]:
df_stacked = df_s.stack(level = 1) 
# 在df.s中，level=0是height...，level=1是Gender
df_stacked.groupby('Class').head(2)

Height  Weight
Class ID   Gender                
C_1   1101 M        173.0    63.0
      1102 F        192.0    73.0
C_2   1201 M        188.0    68.0
      1202 F        176.0    94.0
C_3   1301 M        161.0    68.0
      1302 F        175.0    57.0
C_4   2401 F        192.0    62.0
      2402 M        166.0    82.0

In [54]:
df_stacked = df_s.stack(level=0,dropna=True)
df_stacked.head() # 为啥这个dropna这个参数没有差别？

Gender                 F      M
Class ID                       
C_1   1101 Height    NaN  173.0
           Weight    NaN   63.0
      1102 Height  192.0    NaN
           Weight   73.0    NaN
      1103 Height    NaN  186.0

#### 2) unstack：stack的逆函数，功能上类似于pivot_table
#### 长变宽

In [60]:
result = df_stacked.unstack()\
.swaplevel(1,0,axis=1).sort_index(axis=1)
result.head()

Height        Weight      
Gender          F      M      F     M
Class ID                             
C_1   1101    NaN  173.0    NaN  63.0
      1102  192.0    NaN   73.0   NaN
      1103    NaN  186.0    NaN  82.0
      1104  167.0    NaN   81.0   NaN
      1105  159.0    NaN   64.0   NaN

In [61]:
result.equals(df_s)
#同样在unstack中可以指定level参数

True

In [66]:
df_stacked.unstack(level = 2).head()

Gender          F             M       
           Height Weight Height Weight
Class ID                              
C_1   1101    NaN    NaN  173.0   63.0
      1102  192.0   73.0    NaN    NaN
      1103    NaN    NaN  186.0   82.0
      1104  167.0   81.0    NaN    NaN
      1105  159.0   64.0    NaN    NaN

In [65]:
df_stacked.head()

Gender                 F      M
Class ID                       
C_1   1101 Height    NaN  173.0
           Weight    NaN   63.0
      1102 Height  192.0    NaN
           Weight   73.0    NaN
      1103 Height    NaN  186.0

## 三、哑变量与因子化

### 1. Dummy Variable（哑变量）

重要哦：这里主要介绍get_dummies函数，其功能主要是进行one-hot编码：

In [67]:
df_d = df[['Class','Gender','Weight']]
df_d.head()

,Class,Gender,Weight
0,C_1,M,63
1,C_1,F,73
2,C_1,M,82
3,C_1,F,81
4,C_1,F,64


现在希望将上面的表格前两列转化为哑变量，并加入第三列Weight数值：

In [73]:
pd.get_dummies(df_d[['Class','Gender']],prefix=['try1','try2'])\
 .join(df_d['Weight']).head()
#可选prefix参数添加前缀，prefix_sep添加分隔符

,try1_C_1,try1_C_2,try1_C_3,try1_C_4,try2_F,try2_M,Weight
0,1,0,0,0,0,1,63
1,1,0,0,0,1,0,73
2,1,0,0,0,0,1,82
3,1,0,0,0,1,0,81
4,1,0,0,0,1,0,64


### 2. factorize方法

该方法主要用于自然数编码，并且缺失值会被记做-1，其中sort参数表示是否排序后赋值

In [74]:
codes, uniques = pd.factorize(['b', None, 'a', 'c', 'b'], sort=True)
display(codes)
display(uniques)

array([ 1, -1,  0,  2,  1])

array(['a', 'b', 'c'], dtype=object)

# 问题与练习

1. 上面提到了许多变形函数，如melt/crosstab/pivot/pivot_table/stack/unstack函数，请总结它们各自的使用特点。

长变宽的：pivot/pivot_table/unstacked/crosstable(配合聚合操作服用)

宽变长的：melt/stack

2. 变形函数和多级索引是什么关系？哪些变形函数会使得索引维数变化？具体如何变化？

3. 请举出一个除了上文提过的关于哑变量方法的例子。

In [78]:
pd.get_dummies(df[['School','Gender']]).head()

,School_S_1,School_S_2,Gender_F,Gender_M
0,1,0,0,1
1,1,0,1,0
2,1,0,0,1
3,1,0,1,0
4,1,0,1,0


4. 使用完stack后立即使用unstack一定能保证变化结果与原始表完全一致吗？

不能保证一致，需要经过如去空值，排序等操作

5. 透视表中涉及了三个函数，请分别使用它们完成相同的目标（任务自定）并比较哪个速度最快。

In [123]:
df.pivot(index='ID',columns='Gender',values='Height').agg('mean')


Gender
F    173.473684
M    174.937500
dtype: float64

In [122]:
pd.pivot_table(df,index='School',columns='Gender',values='Height',aggfunc='mean').agg('mean')

Gender
F    173.426136
M    175.357143
dtype: float64

In [119]:
pd.crosstab(index=df['School'],columns=df['Gender'],
            values=df['Height'],aggfunc='mean',margins=True).loc['All']

Gender
F      173.473684
M      174.937500
All    174.142857
Name: All, dtype: float64

In [125]:
%timeit df.pivot(index='ID',columns='Gender',values='Height').agg('mean')
%timeit pd.pivot_table(df,index='School',columns='Gender',values='Height',aggfunc='mean').agg('mean')
%timeit pd.crosstab(index=df['School'],columns=df['Gender'],values=df['Height'],aggfunc='mean',margins=True).loc['All']

1.85 ms ± 80.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
7.35 ms ± 297 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
23 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


6. 既然melt起到了stack的功能，为什么再设计stack函数？

我也在思考这个问题，pivot和unstacked也是同理

In [140]:
df = pd.read_csv('data/Drugs.csv')
df.head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5


In [165]:
df = pd.read_csv('data/Drugs.csv',index_col=['State','COUNTY','SubstanceName']).sort_index()
df.head()

YYYY  DrugReports
State COUNTY SubstanceName                   
KY    ADAIR  Buprenorphine  2011            3
             Buprenorphine  2012            5
             Buprenorphine  2013            4
             Buprenorphine  2014           27
             Buprenorphine  2015            5

In [158]:
df1=pd.pivot_table(df,index=['State','COUNTY','SubstanceName'],
               columns='YYYY',values='DrugReports',fill_value='-')\
.sort_index().reset_index().rename_axis(columns={'YYYY':''})
df1.head()

,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,-,3,5,4,27,5,7,10
1,KY,ADAIR,Codeine,-,-,1,-,-,-,-,1
2,KY,ADAIR,Fentanyl,-,-,1,-,-,-,-,-
3,KY,ADAIR,Heroin,-,-,1,2,-,1,-,2
4,KY,ADAIR,Hydrocodone,6,9,10,10,9,7,11,3


b) 现在请将(a)中的结果恢复到原数据表，并通过equal函数检验初始表与新的结果是否一致（返回True）

参考答案：

In [160]:
result = df1
result_melted = result.melt(id_vars=result.columns[:3],value_vars=result.columns[-8:]
                ,var_name='YYYY',value_name='DrugReports').query('DrugReports != "-"')
result2 = result_melted.sort_values(by=['State','COUNTY','YYYY'
                                    ,'SubstanceName']).reset_index().drop(columns='index')
#下面其实无关紧要，只是交换两个列再改一下类型（因为‘-’所以type变成object了）
cols = list(result2.columns)
a, b = cols.index('SubstanceName'), cols.index('YYYY')
cols[b], cols[a] = cols[a], cols[b]
result2 = result2[cols].astype({'DrugReports':'int','YYYY':'int'})
result2.head()

,State,COUNTY,YYYY,SubstanceName,DrugReports
0,KY,ADAIR,2010,Hydrocodone,6
1,KY,ADAIR,2010,Methadone,1
2,KY,ADAIR,2011,Buprenorphine,3
3,KY,ADAIR,2011,Hydrocodone,9
4,KY,ADAIR,2011,Morphine,2


In [168]:
df_tidy = df.reset_index().sort_values(by=result2.columns[:4].tolist()).reset_index().drop(columns='index')
df_tidy.head()

,State,COUNTY,SubstanceName,YYYY,DrugReports
0,KY,ADAIR,Hydrocodone,2010,6
1,KY,ADAIR,Methadone,2010,1
2,KY,ADAIR,Buprenorphine,2011,3
3,KY,ADAIR,Hydrocodone,2011,9
4,KY,ADAIR,Morphine,2011,2


In [169]:
df_tidy.equals(result2) #有两列顺序不一样 

False

In [176]:
df_eq=pd.read_csv('data/Earthquake.csv')
df_eq.head()

,日期,时间,维度,经度,方向,距离,深度,烈度
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


In [177]:
result = pd.pivot_table(df_eq,index=['日期','时间','维度','经度']
            ,columns='方向'
            ,values=['烈度','深度','距离'],fill_value='-').rename_axis(index={None:'地震参数'})
result.head(6)

深度                                    \
方向                               east north north_east north_west south   
日期         时间          维度   经度                                            
1912.08.09 12:29:00 AM 40.6 27.2    -     -          -          -     -   
1912.08.10 12:23:00 AM 40.6 27.1    -     -          -          -     -   
           12:30:00 AM 40.6 27.1    -     -          -          -     -   
1912.08.11 12:19:04 AM 40.6 27.2    -     -          -          -     -   
           12:20:00 AM 40.6 27.1    -     -          -          -     -   
1912.09.16 12:04:02 AM 40.1 26.8    -     -         30          -     -   

                                                              烈度        ...  \
方向                               south_east south_west west east north  ...   
日期         时间          维度   经度                                          ...   
1912.08.09 12:29:00 AM 40.6 27.2         16          -    -    -     -  ...   
1912.08.10 12:23:00 AM 40.6 27.1          -         15    -    -     -  ...   
           12:30:00 AM 40.6 27.1          -         15    -    -     -  ...   
1912.08.11 12:19:04 AM 40.6 27.2         30          -    -    -     -  ...   
           12:20:00 AM 40.6 27.1          -         15    -    -     -  ...   
1912.09.16 12:04:02 AM 40.1 26.8          -          -    -    -     -  ...   

                                                   距离                   \
方向                               south_west west east north north_east   
日期         时间          维度   经度                                           
1912.08.09 12:29:00 AM 40.6 27.2          -    -    -     -          -   
1912.08.10 12:23:00 AM 40.6 27.1          6    -    -     -          -   
           12:30:00 AM 40.6 27.1        5.2    -    -     -          -   
1912.08.11 12:19:04 AM 40.6 27.2          -    -    -     -          -   
           12:20:00 AM 40.6 27.1        4.5    -    -     -          -   
1912.09.16 12:04:02 AM 40.1 26.8          -    -    -     -        4.3   

                                                                              
方向                               north_west south south_east south_west west  
日期         时间          维度   经度                                                
1912.08.09 12:29:00 AM 40.6 27.2          -     -        4.3          -    -  
1912.08.10 12:23:00 AM 40.6 27.1          -     -          -          2    -  
           12:30:00 AM 40.6 27.1          -     -          -          2    -  
1912.08.11 12:19:04 AM 40.6 27.2          -     -        4.3          -    -  
           12:20:00 AM 40.6 27.1          -     -          -          2    -  
1912.09.16 12:04:02 AM 40.1 26.8          -     -          -          -    -  

[6 rows x 24 columns]

正确答案中有stack参数

In [179]:
result = pd.pivot_table(df_eq,index=['日期','时间','维度','经度']
            ,columns='方向'
            ,values=['烈度','深度','距离'],fill_value='-').stack(level=0).rename_axis(index={None:'地震参数'})
result.head(6)

方向                                    east north north_east north_west south  \
日期         时间          维度   经度   地震参数                                          
1912.08.09 12:29:00 AM 40.6 27.2 深度      -     -          -          -     -   
                                 烈度      -     -          -          -     -   
                                 距离      -     -          -          -     -   
1912.08.10 12:23:00 AM 40.6 27.1 深度      -     -          -          -     -   
                                 烈度      -     -          -          -     -   
                                 距离      -     -          -          -     -   

方向                                    south_east south_west west  
日期         时间          维度   经度   地震参数                             
1912.08.09 12:29:00 AM 40.6 27.2 深度           16          -    -  
                                 烈度          6.7          -    -  
                                 距离          4.3          -    -  
1912.08.10 12:23:00 AM 40.6 27.1 深度            -         15    -  
                                 烈度            -          6    -  
                                 距离            -          2    -